In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

import tensorflow.keras as tf

In [ ]:
books_df = pd.read_csv('/content/drive/My Drive/books2.csv')
ratings_df = pd.read_csv("/content/drive/My Drive/ratings.csv") 

In [ ]:
books_df.head()
ratings_df.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest = train_test_split(ratings_df, test_size=0.2, random_state=1)
Xtrain.shape
Xtest.shape

Shape of train data: (4781183, 3)
Shape of test data: (1195296, 3)


In [ ]:
nbook_id = max(ratings_df.book_id)
nuser_id = max(ratings_df.user_id)

53424


In [ ]:
#books
books_input = tf.layers.Input(shape=[1])
embedding_books = tf.layers.Embedding(nbook_id + 500,13)(books_input)
books_out = tf.layers.Flatten()(embedding_books)

#users
users_input = tf.layers.Input(shape=[1])
embedding_users = tf.layers.Embedding(nuser_id + 500,13)(users_input)
users_out = tf.layers.Flatten()(embedding_users)

concatenated_layer = tf.layers.Concatenate()([books_out, users_out])
hidden_layer = tf.layers.Dense(128, activation='relu')(concatenated_layer)
output_layer = hidden_layer = tf.layers.Dense(1, activation='relu')(hidden_layer)
model = tf.Model([books_input, users_input], output_layer)

Tensor("input_14:0", shape=(None, 1), dtype=float32)


In [ ]:
#loading the model
model = tf.models.load_model('/content/drive/My Drive/the_model')

In [ ]:
opt = tf.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='mean_squared_error')
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 13)        136500      input_5[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 1, 13)        701012      input_6[0][0]                    
_______________________________________________________________________________________

In [ ]:
hist = model.fit([Xtrain.book_id, Xtrain.user_id], Xtrain.rating, 
                 batch_size=64, 
                 epochs=5, 
                 verbose=1,
                 validation_data=([Xtest.book_id, Xtest.user_id], Xtest.rating))

Epoch 1/5
74706/74706 [==============================] - 790s 11ms/step - loss: 0.7809 - val_loss: 0.7339
Epoch 2/5
74706/74706 [==============================] - 782s 10ms/step - loss: 0.7121 - val_loss: 0.7069
Epoch 3/5
74706/74706 [==============================] - 767s 10ms/step - loss: 0.6790 - val_loss: 0.6931
Epoch 4/5
74706/74706 [==============================] - 747s 10ms/step - loss: 0.6556 - val_loss: 0.6845
Epoch 5/5
74706/74706 [==============================] - 746s 10ms/step - loss: 0.6378 - val_loss: 0.6813


In [ ]:
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
plt.plot(train_loss, color='g', label='Train Loss')
plt.plot(val_loss, color='b', label='Validation Loss')
plt.title("Train and Validation Loss Curve")
plt.legend()
plt.show()

In [ ]:
#saving the model
model.save("/content/drive/My Drive/the_model")

In [45]:
books_df_copy = books_df.copy()
books_df_copy = books_df_copy.set_index("book_id")

In [46]:
#making recommendations for user with id 3000
book_array = np.array(unique_books_ids)
user = np.array([3000 for i in range(len(unique_books_ids))])
prediction = model.predict([book_array, user])
prediction

array([[4.3927407],
       [3.3497446],
       [3.7533956],
       ...,
       [3.5077271],
       [4.028384 ],
       [3.588623 ]], dtype=float32)

In [47]:
prediction = prediction.reshape(-1) 
prediction_ids = (-prediction).argsort()[0:5]
prediction_ids

array([ 479,  973, 3260, 1830, 1534])

In [48]:
books_df.iloc[prediction_ids]

,book_id,authors,original_publication_year,title,average_rating,ratings_count,image_url
479,508,Barbara Ehrenreich,2001.0,Nickel and Dimed: On (Not) Getting By in America,3.60,152786,https://s.gr-assets.com/assets/nophoto/book/11...
973,1045,Veronica Roth,2014.0,"Four: A Divergent Story Collection (Divergent,...",4.10,97058,https://images.gr-assets.com/books/1393687572m...
3260,3772,Herman Wouk,1978.0,"War and Remembrance (The Henry Family, #2)",4.38,26633,https://s.gr-assets.com/assets/nophoto/book/11...
1830,2021,Stephenie Meyer,2007.0,"The Twilight Collection (Twilight, #1-3)",3.78,42361,https://s.gr-assets.com/assets/nophoto/book/11...
1534,1686,"David Eddings, Leigh Eddings",1995.0,Belgarath the Sorcerer,4.07,59737,https://images.gr-assets.com/books/1333692194m...
